In [ ]:
import numpy as np
from scipy.optimize import minimize
import numpy as np
from collections import defaultdict
import math
from functools import reduce
import json


def enhanced_scale(arr, scale_min=1, scale_max=10):
    """
    改进的归一化函数：每个维度独立标准化，再缩放到1-10之间
    """
    arr = np.array(arr)
    scaled = np.zeros_like(arr)
    
    for i in range(arr.shape[1]):
        col = arr[:, i]
        # 标准化：均值为0，方差为1
        col_standardized = (col - np.mean(col)) / np.std(col)
        # 缩放到1-10
        scaled[:, i] = scale_min + (col_standardized - np.min(col_standardized)) * (scale_max - scale_min) / (np.max(col_standardized) - np.min(col_standardized))
    
    return np.round(scaled).astype(int)  # 取整保证整数系数

def model_coefficients(x, coeffs):
    """
    计算多项式的输出
    """
    a, b, c, d, e, f, g, h, i = coeffs
    x1, x2, x3 = x
    return a*x1**2 + b*x2**2 + c*x3**2 + d*x1*x2 + e*x1*x3 + f*x2*x3 + g*x1 + h*x2 + i*x3

def objective(coeffs, best, others):
    """
    目标函数：最大化best的y值，同时最小化others的y值
    """
    y_best = model_coefficients(best, coeffs)
    y_others = np.array([model_coefficients(other, coeffs) for other in others])
    
    # 使得others中的y值小于y_best
    penalty = np.sum(np.maximum(0, y_others - y_best) ** 2)  # 强化惩罚：平方罚项
    return -y_best + penalty  # 最小化目标函数，负号表示最大化best的y

def find_optimal_coeffs(best, others):
    """
    使用约束优化算法寻找最优的系数a~i
    """
    initial_guess = np.ones(9)  # 初始猜测系数为1
    bounds = [(-10, 10)] * 9  # 限制系数在-10到10的范围内
    
    # 约束条件：确保y_other < y_best
    def constraint(coeffs):
        y_best = model_coefficients(best, coeffs)
        y_others = np.array([model_coefficients(other, coeffs) for other in others])
        return np.min(y_best - y_others)  # 确保所有y_other都小于y_best
    
    cons = ({'type': 'ineq', 'fun': constraint})  # 约束条件，确保y_others < y_best
    
    result = minimize(objective, initial_guess, args=(best, others), method='trust-constr', bounds=bounds, constraints=cons)
    return result.x

def validate(coeffs, best, others):
    """
    验证是否所有other的y值都小于best的y值
    """
    y_best = model_coefficients(best, coeffs)
    y_others = np.array([model_coefficients(other, coeffs) for other in others])
    for index,other in enumerate(y_others):
        if y_best < other:
            print(others[index])
            print('best:{}\tother:{}'.format(y_best, other))
    # 检查是否所有y_others都小于y_best
    valid = np.all(y_others <= y_best)
    
    return valid, y_best, y_others


In [ ]:
res = {}

In [ ]:
with open('./count_datas/f1_coefficient_train.json', 'r') as f:
    tmp_datas = json.load(f)
res['f1'] = tmp_datas


In [ ]:
now_type = 'f1'
# now_type = '算力'
# now_type = '时间'

now_key = 'CMeIE---带实体类型'
# now_key = 'CMeIE---不带实体类型'
# now_key = 'ADE---带实体类型'
# now_key = 'ADE---不带实体类型'

In [ ]:
best_str_list = res[now_type][now_key]['best_str_list']
other_str_list = res[now_type][now_key]['other_str_list']

In [ ]:
best_str_list[1]

In [ ]:
for best_str, other_str in zip(best_str_list, other_str_list):
    best = best_str.split('\t')
    best = [float(item) for item in best]
    others = []
    for item in other_str.split('\n'):
        if item == '':
            continue
        tmp_list = item.split('\t')
        tmp_list = [float(_) for _ in tmp_list]
        others.append(tmp_list)
    
    # 删除best那组数据
    others = [other for other in others if not np.allclose(other, best)]
    print(len(others))
    # 合并best和others
    all_solutions = [best] + others

    # 归一化处理所有数据
    processed_solutions = enhanced_scale(all_solutions, scale_min=1, scale_max=10)

    # 提取归一化后的best和others
    processed_best = processed_solutions[0]
    processed_others = processed_solutions[1:]

    # 使用优化算法找到最佳系数
    optimal_coeffs = find_optimal_coeffs(processed_best, processed_others)


    print("Optimal coefficients:", optimal_coeffs)

    # 验证是否所有other的y值都小于best的y值
    valid, y_best, y_others = validate(optimal_coeffs, processed_best, processed_others)

    # 输出验证结果
    if valid:
        print("验证成功！所有other的y值都小于best的y值。")
    else:
        print("验证失败！以下y值不符合要求：")
        print("y_best:", y_best)
        print("y_others:", y_others)

In [ ]:
best_str_list

In [ ]:
# CMeIE---带实体类型
# optimal_coeffs = [9, -10, -7.5, 7, 10, 3, 10, 10, 10]

# CMeIE---不带实体类型
optimal_coeffs = [10 , -8, -8, 10, 10, 4, 10, -10, 7]

# ADE---带实体类型
# optimal_coeffs = [-8.5 , -9, 5, 6.5, 10, 9, 2, 10, 7]

# ADE---不带实体类型
# optimal_coeffs = [10 , -10, -10, 10, 10, 7.5, 10, 10, 3]

In [ ]:
best_str_list

In [ ]:
for best_str, other_str in zip(best_str_list, other_str_list):
    best = best_str.split('\t')
    best = [float(item) for item in best]
    others = []
    for item in other_str.split('\n'):
        if item == '':
            continue
        tmp_list = item.split('\t')
        tmp_list = [float(_) for _ in tmp_list]
        others.append(tmp_list)
    
    # 删除best那组数据
    others = [other for other in others if not np.allclose(other, best)]
    # 合并best和others
    all_solutions = [best] + others

    
    # 归一化处理所有数据
    processed_solutions = enhanced_scale(all_solutions, scale_min=1, scale_max=10)

    processed_best = processed_solutions[0]
    processed_others = processed_solutions[1:]



    print(processed_best)
    print("Optimal coefficients:", optimal_coeffs)

    # 验证是否所有other的y值都小于best的y值
    valid, y_best, y_others = validate(optimal_coeffs, processed_best, processed_others)

    # 输出验证结果
    if valid:
        print("验证成功！所有other的y值都小于best的y值。")
    else:
        print("验证失败！以上y值不符合要求\n\n")
        # print("y_best:", y_best)
        # print("y_others:", y_others)